In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pandas as pd
import numpy as np
import seaborn as sns
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize 
from sklearn.multioutput import MultiOutputClassifier
from skmultilearn.problem_transform import ClassifierChain
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
main_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip')
test_df = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip')
test_labels = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip')
sample_submission = pd.read_csv('/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip')

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


/kaggle/input/sample_submission.csv
/kaggle/input/test.csv.zip
/kaggle/input/train.csv.zip
/kaggle/input/description.md
/kaggle/input/sample_submission.csv.zip
/kaggle/input/train.csv
/kaggle/input/test.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/description.md
/kaggle/input/jigsaw-toxic-comment-classification-challenge/sample_submission.csv.zip
/kaggle/input/jigsaw-toxic-comment-classification-challenge/train.csv
/kaggle/input/jigsaw-toxic-comment-classification-challenge/test.csv


FileNotFoundError: [Errno 2] No such file or directory: '/kaggle/input/jigsaw-toxic-comment-classification-challenge/test_labels.csv.zip'

In [3]:
main_df.head()


,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [4]:
test_df.head()


,id,comment_text
0,00001cee341fdb12,Yo bitch Ja Rule is more succesful then you'll...
1,0000247867823ef7,== From RfC == \n\n The title is fine as it is...
2,00013b17ad220c46,""" \n\n == Sources == \n\n * Zawe Ashton on Lap..."
3,00017563c3f7919a,":If you have a look back at the source, the in..."
4,00017695ad8997eb,I don't anonymously edit articles at all.


In [5]:
#data cleaning
def data_cleaning(df):
    df['comment_text'] = df['comment_text'].replace('\n',' ', regex=True).replace('\r',' ', regex=True).replace('\t',' ', regex=True) #spaces
    df['comment_text'] = df['comment_text'].replace('\\n',' ', regex=True)
    df['comment_text'] = df['comment_text'].replace('http[S|s]*\S+|www.\S+',' ', regex=True) #Links
    df['comment_text'] = df['comment_text'].replace('\d+',' ', regex=True) #numbers
    df['comment_text'] = df['comment_text'].replace('\s{2,}',' ', regex=True) #multiple spaces
    df['comment_text'] = df['comment_text'].map(lambda x: x.strip()) #removing trailSpaces
    df['comment_text'] = df['comment_text'].str.replace('\W', ' ') #special characters => most of the puntuat
    return df


In [6]:
main_df = data_cleaning(main_df)
test_df = data_cleaning(test_df)


In [7]:
#tokenization
#def tokenizer(text):
#    return word_tokenize(text)
#print(stopwords.words('english'))
new_stopwords = stopwords.words('english')

def stopwords_remover(words_list):
    text_list = [word for word in words_list if word not in new_stopwords]
    return text_list

lemmitize_obj = nltk.WordNetLemmatizer()

def _lemmatizer(tokenized_text):
    text = [lemmitize_obj.lemmatize(word) for word in tokenized_text]
    return text


In [8]:
main_df['comment_text_tokenized'] = main_df['comment_text'].apply(lambda x: word_tokenize(x.lower()))
main_df['comment_text_stopword_removed'] = main_df['comment_text_tokenized'].apply(lambda x: stopwords_remover(x))
main_df['comment_text_lemmatized'] = main_df['comment_text_stopword_removed'].apply(lambda x: _lemmatizer(x))

test_df['comment_text_tokenized'] = test_df['comment_text'].apply(lambda x: word_tokenize(x.lower()))
test_df['comment_text_stopword_removed'] = test_df['comment_text_tokenized'].apply(lambda x: stopwords_remover(x))
test_df['comment_text_lemmatized'] = test_df['comment_text_stopword_removed'].apply(lambda x: _lemmatizer(x))

wrk_df = main_df.copy()
wrk_df =  wrk_df.drop(['comment_text','comment_text_stopword_removed','comment_text_tokenized'],axis = 1)
wrk_df['comment_text_lemmatized']= wrk_df['comment_text_lemmatized'].map(lambda x: " ".join(x) ) 
test_df['comment_text_lemmatized']= test_df['comment_text_lemmatized'].map(lambda x: " ".join(x) )


In [9]:
tf_vect = TfidfVectorizer(lowercase = True, max_features=1750)


In [10]:
train_features_sparse = tf_vect.fit_transform(wrk_df['comment_text_lemmatized'])
test_features_sparse = tf_vect.transform(test_df['comment_text_lemmatized'])


In [11]:
test_df['comment_text_lemmatized'].head()


0    yo bitch ja rule succesful 'll ever whats hati...
1                         == rfc == title fine , imo .
2         `` == source == * zawe ashton lapland — / ``
3    : look back source , information updated corre...
4                       n't anonymously edit article .
Name: comment_text_lemmatized, dtype: object

In [12]:
clf = ClassifierChain(MultinomialNB(fit_prior=True)).fit(train_features_sparse, wrk_df[['toxic','severe_toxic','obscene','threat','insult','identity_hate']])


In [13]:
y_predict = clf.predict(test_features_sparse)


In [14]:
y_predict_proba = clf.predict_proba(test_features_sparse)


In [15]:
predict_dense = y_predict_proba.todense()
print(predict_dense)


[[9.89707236e-01 6.60248092e-01 9.99978532e-01 8.55857092e-01
  9.99999608e-01 9.99884561e-01]
 [1.61785837e-02 1.41398033e-03 4.21641733e-03 1.14283687e-03
  4.56159575e-03 1.18098321e-03]
 [3.31217095e-02 1.03970366e-03 1.09709262e-02 1.97520235e-04
  7.83854340e-03 7.69987947e-04]
 ...
 [5.83857654e-03 3.67129707e-04 1.12053756e-03 3.89018710e-04
  5.31608870e-04 4.67248880e-04]
 [5.40069732e-02 1.37079325e-03 5.29434874e-03 2.48425726e-03
  3.23317838e-03 4.11729803e-03]
 [5.80353826e-01 2.56994975e-02 7.06273194e-01 4.66987325e-02
  9.65663419e-01 5.70363672e-01]]


In [16]:
data_df = pd.DataFrame(predict_dense,columns= ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate'])

submission_df_1 = pd.concat([test_df['id'], data_df] ,axis = 1)


In [17]:
submission_df_1.head()


,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,00001cee341fdb12,0.989707,0.660248,0.999979,0.855857,1.000000,0.999885
1,0000247867823ef7,0.016179,0.001414,0.004216,0.001143,0.004562,0.001181
2,00013b17ad220c46,0.033122,0.001040,0.010971,0.000198,0.007839,0.000770
3,00017563c3f7919a,0.009800,0.000492,0.001621,0.000224,0.000932,0.000177
4,00017695ad8997eb,0.042644,0.000927,0.012771,0.000245,0.008294,0.000652


In [18]:
submission_df_1.to_csv('submission.csv',index=False)
